In [1]:
import pandas as pd
import requests

In [2]:
pd.options.display.max_columns = 24

### dProducts

In [3]:
r = requests.get('https://fakestoreapi.com/products').json()
dim_products_raw = pd.DataFrame.from_dict(r)
expanded_dim_products = pd.DataFrame(dim_products_raw['rating'].to_list())
dim_products = pd.merge(dim_products_raw, expanded_dim_products, left_index=True, right_index=True, how='left')
dim_products.drop(columns=['rating'], inplace=True)
dim_products.set_index('id', drop=True, inplace=True)
dim_products.head()

,title,price,description,category,image,rate,count
id,,,,,,,
1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",109.95,Your perfect pack for everyday use and walks i...,men's clothing,https://fakestoreapi.com/img/81fPKd-2AYL._AC_S...,3.9,120
2,Mens Casual Premium Slim Fit T-Shirts,22.30,"Slim-fitting style, contrast raglan long sleev...",men's clothing,https://fakestoreapi.com/img/71-3HjGNDUL._AC_S...,4.1,259
3,Mens Cotton Jacket,55.99,great outerwear jackets for Spring/Autumn/Wint...,men's clothing,https://fakestoreapi.com/img/71li-ujtlUL._AC_U...,4.7,500
4,Mens Casual Slim Fit,15.99,The color could be slightly different between ...,men's clothing,https://fakestoreapi.com/img/71YXzeOuslL._AC_U...,2.1,430
5,John Hardy Women's Legends Naga Gold & Silver ...,695.00,"From our Legends Collection, the Naga was insp...",jewelery,https://fakestoreapi.com/img/71pWzhdJNwL._AC_U...,4.6,400


### fCart

In [4]:
r1 = requests.get('https://fakestoreapi.com/carts').json()
fat_cart_raw = pd.DataFrame.from_dict(r1)
fat_cart_expl = fat_cart_raw.explode(column='products')
fat_cart_expl.reset_index(inplace=True, drop=True)
expanded_fat_cart = pd.DataFrame(fat_cart_expl['products'].to_list())
fat_cart = pd.merge(fat_cart_expl, expanded_fat_cart, left_index=True, right_index=True, how='left')
fat_cart.drop(columns=['products', '__v'], inplace=True)
fat_cart.head()

,id,userId,date,productId,quantity
0,1,1,2020-03-02T00:00:00.000Z,1,4
1,1,1,2020-03-02T00:00:00.000Z,2,1
2,1,1,2020-03-02T00:00:00.000Z,3,6
3,2,1,2020-01-02T00:00:00.000Z,2,4
4,2,1,2020-01-02T00:00:00.000Z,1,10


### dUser

In [5]:
r2 = requests.get('https://fakestoreapi.com/users').json()
dim_user_raw = pd.DataFrame.from_dict(r2)
#
expanded_address = pd.DataFrame(dim_user_raw["address"].to_list())
expanded_address_2 = pd.DataFrame(expanded_address["geolocation"].to_list())
dim_user_address = pd.merge(expanded_address, expanded_address_2, left_index=True, right_index=True)
dim_user_address.drop(columns=["geolocation"], inplace=True)
#
expanded_name = pd.DataFrame(dim_user_raw["name"].to_list())
dim_user = pd.merge(dim_user_raw, dim_user_address, left_index=True, right_index=True)
dim_user.drop(columns="address",inplace=True)
dim_user = pd.merge(dim_user, expanded_name, left_index=True, right_index=True)
dim_user.drop(columns=["name", "__v"], inplace=True)
dim_user.set_index("id", drop=True, inplace=True)
#
dim_user.head()

,email,username,password,phone,city,street,number,zipcode,lat,long,firstname,lastname
id,,,,,,,,,,,,
1,john@gmail.com,johnd,m38rmF$,1-570-236-7033,kilcoole,new road,7682,12926-3874,-37.3159,81.1496,john,doe
2,morrison@gmail.com,mor_2314,83r5^_,1-570-236-7033,kilcoole,Lovers Ln,7267,12926-3874,-37.3159,81.1496,david,morrison
3,kevin@gmail.com,kevinryan,kev02937@,1-567-094-1345,Cullman,Frances Ct,86,29567-1452,40.3467,-30.1310,kevin,ryan
4,don@gmail.com,donero,ewedon,1-765-789-6734,San Antonio,Hunters Creek Dr,6454,98234-1734,50.3467,-20.1310,don,romer
5,derek@gmail.com,derek,jklg*_56,1-956-001-1945,san Antonio,adams St,245,80796-1234,40.3467,-40.1310,derek,powell


### Aggregated db

In [6]:
full_db = pd.merge(fat_cart, dim_products, how = "left", left_on = "productId", right_on = "id")
full_db = pd.merge(full_db, dim_user, how = "left", left_on = "userId", right_on = "id")
full_db.rename({"id":"cartId"})
full_db.head()

,id,userId,date,productId,quantity,title,price,description,category,image,rate,count,email,username,password,phone,city,street,number,zipcode,lat,long,firstname,lastname
0,1,1,2020-03-02T00:00:00.000Z,1,4,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",109.95,Your perfect pack for everyday use and walks i...,men's clothing,https://fakestoreapi.com/img/81fPKd-2AYL._AC_S...,3.9,120,john@gmail.com,johnd,m38rmF$,1-570-236-7033,kilcoole,new road,7682,12926-3874,-37.3159,81.1496,john,doe
1,1,1,2020-03-02T00:00:00.000Z,2,1,Mens Casual Premium Slim Fit T-Shirts,22.30,"Slim-fitting style, contrast raglan long sleev...",men's clothing,https://fakestoreapi.com/img/71-3HjGNDUL._AC_S...,4.1,259,john@gmail.com,johnd,m38rmF$,1-570-236-7033,kilcoole,new road,7682,12926-3874,-37.3159,81.1496,john,doe
2,1,1,2020-03-02T00:00:00.000Z,3,6,Mens Cotton Jacket,55.99,great outerwear jackets for Spring/Autumn/Wint...,men's clothing,https://fakestoreapi.com/img/71li-ujtlUL._AC_U...,4.7,500,john@gmail.com,johnd,m38rmF$,1-570-236-7033,kilcoole,new road,7682,12926-3874,-37.3159,81.1496,john,doe
3,2,1,2020-01-02T00:00:00.000Z,2,4,Mens Casual Premium Slim Fit T-Shirts,22.30,"Slim-fitting style, contrast raglan long sleev...",men's clothing,https://fakestoreapi.com/img/71-3HjGNDUL._AC_S...,4.1,259,john@gmail.com,johnd,m38rmF$,1-570-236-7033,kilcoole,new road,7682,12926-3874,-37.3159,81.1496,john,doe
4,2,1,2020-01-02T00:00:00.000Z,1,10,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",109.95,Your perfect pack for everyday use and walks i...,men's clothing,https://fakestoreapi.com/img/81fPKd-2AYL._AC_S...,3.9,120,john@gmail.com,johnd,m38rmF$,1-570-236-7033,kilcoole,new road,7682,12926-3874,-37.3159,81.1496,john,doe
